# **Import needed modules**

In [ ]:
import os
import pandas as pd
from fastai.vision.all import *
from fastai.vision import models
from fastai.metrics import accuracy, F1Score

import warnings
warnings.filterwarnings("ignore")
set_seed(42)

print ('modules loaded')

# **Data Preprocessing**

#### **Read data and store it in dataframe**

In [ ]:
# Generate data paths with labels
data_dir = '../input/mango-leaf-disease-dataset'
filepaths = []
labels = []

folds = os.listdir(data_dir)
for fold in folds:
    foldpath = os.path.join(data_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name= 'filepaths')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis= 1)

In [ ]:
df

In [ ]:
dls = ImageDataLoaders.from_df(df,
                                fn_col=0, # filepaths
                                label_col=1, # labels
                                valid_pct=0.2,
                                folder='', 
                                item_tfms=Resize(224))

In [ ]:
dls.show_batch(max_n=16)

# **Model Structure**

In [ ]:
learn = vision_learner(dls, 'efficientnet_b0', metrics=[accuracy], path='.').to_fp16()
learn.summary()

## **Training**

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.show_results()

In [ ]:
# Save the model
learn.export('/kaggle/working/mango_diseases.pkl')

In [ ]:
# Build a Classification Interpretation object from our learn model
# it can show us where the model made the worse predictions:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
# Plot the top ‘n’ classes where the classifier has least precision.
interp.plot_top_losses(12, figsize=(15, 15))

In [ ]:
interp.plot_confusion_matrix(figsize=(12, 12), dpi=60)

In [ ]:
#To view the list of classes most misclassified as a list
interp.most_confused(min_val=2) #We are ignoring single image misclassification

## **Thank You..**
If you find this notebook is good enough, please upvote it..